# Tarea Hogar Número 1

 ## Punto 5
 
Encontrar los mejores hiperparámetros del árbol de decisión rpart para nuestro problema: método Grid Search

Leer el [artículo](https://www.learnbymarketing.com/tutorials/rpart-decision-trees-in-r/ "Title")

En el script rpart/z241_gridsearch_esqueleto.r (está en el repositorio GitHub de la asignatura) se encuentra un humilde esqueleto de Grid Search sobre solamente dos de los hiperparámetros del árbol < min_split, max_depth >
Su misión, si decide aceptarla, será:

+ modificar el script para que itere sobre TODOS los hiperparámetros de rpart
+ correrlo por 24 horas, hasta encontrar los mejores hiperparámetros
+ analizar la salida que se genera, viendo si existe la posibilidad de hacer una segunda corrida pero sobre una región más pequeña del espacio de búsqueda lo que fue mencionado en clase como "sintonía fina".
+ para los mejores hiperparámetros que encuentre, pruebe agregarlos al script rpart/z242_aplicar_rpart_kaggle.r generar la salida para Kaggle, y subirla, a ver si puede destronar al ganador.


La tarea de modificar el script para considerar todos los hiperparámetros es conceptualmente más estratégica de lo que parece, ya que minsplit y minbucket están relacionados entre sí mediante la fórmula 2*minbucket <= minsplit (no puede suceder que ambos hijos de un nodo poseen más de la mitad de los registros del padre)

Comentar en Zulip los hiperparámetros buenos que va encontrando. Compartiendo no estará revelando la Magic Sauce a sus compañeros ya que el algoritmo que va a ganar la competencia será en Ensemble de LightGBM, no un infeliz árbol de decisión.

Si cualquiera de los miembros de su equipo padeciera un colapso nervioso y/o angustia existencial ante la variablidad de los resultados, este profesor negará cualquier responsabilidad sobre el asunto ante el decano de la facultad.

## SCRIPT

In [7]:
#esqueleto de grid search
#se espera que los alumnos completen lo que falta para recorrer TODOS cuatro los hiperparametros 

rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

require("data.table")
require("rpart")
require("parallel")

ksemillas  <- c(333333, 343434, 353535, 363636, 373737) #reemplazar por las propias semillas

# Definido por gustavo la función particionar para hacer los dataset de entrenamiento y testeo
#------------------------------------------------------------------------------
# particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 

particionar  <- function( data,  division, agrupa="",  campo="fold", start=1, seed=NA )
{
  if( !is.na(seed) )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x )} ,   division,  seq( from=start, length.out=length(division) )  ) )  

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
          by= agrupa ]
}

# Qué hace exactamente el if este?
#------------------------------------------------------------------------------

ArbolEstimarGanancia  <- function( semilla, param_basicos )
{
  #particiono estratificadamente el dataset
  particionar( dataset, division=c(70,30), agrupa="clase_ternaria", seed = 333333 )  # Cambiar por la primer semilla de cada uno !

  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                   data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                   xval= 0,
                   control= param_basicos )  #aqui van los parametros del arbol

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo,   #el modelo que genere recien
                          dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                          type= "prob") #type= "prob"  es que devuelva la probabilidad

  #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  #cada columna es el vector de probabilidades 


  #calculo la ganancia en testing que es fold==2
  ganancia_test  <- dataset[ fold==2, 
                             sum( ifelse( prediccion[, "BAJA+2"]  >  1/60,
                                         ifelse( clase_ternaria=="BAJA+2", 59000, -1000 ),
                                         0 ) )]

  #escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada  <-  ganancia_test / 0.3

  return( ganancia_test_normalizada )
}
#------------------------------------------------------------------------------

ArbolesMontecarlo  <- function( semillas, param_basicos )
{
  #la funcion mcmapply  llama a la funcion ArbolEstimarGanancia  tantas veces como valores tenga el vector  ksemillas
  ganancias  <- mcmapply( ArbolEstimarGanancia, 
                          semillas,   #paso el vector de semillas, que debe ser el primer parametro de la funcion ArbolEstimarGanancia
                          MoreArgs= list( param_basicos),  #aqui paso el segundo parametro
                          SIMPLIFY= FALSE,
                          mc.cores= 5 )  #se puede subir a 5 si posee Linux o Mac OS

  ganancia_promedio  <- mean( unlist(ganancias) )

  return( ganancia_promedio )
}
#------------------------------------------------------------------------------

#Aqui se debe poner la carpeta de la computadora local
setwd("~/Documents/MDL")   #Establezco el Working Directory

#cargo los datos
dataset  <- fread("./datasets/paquete_premium_202011.csv")


#genero el archivo para Kaggle
#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./labo/exp/",  showWarnings = FALSE ) 
dir.create( "./labo/exp/HT2020/", showWarnings = FALSE )
archivo_salida  <- "./labo/exp/HT2020/gridsearch.txt"

#Escribo los titulos al archivo donde van a quedar los resultados
#atencion que si ya existe el archivo, esta instruccion LO SOBREESCRIBE, y lo que estaba antes se pierde
#la forma que no suceda lo anterior es con append=TRUE
cat( file=archivo_salida,
     sep= "",
     "max_depth", "\t",
     "min_split", "\t",
     "cp", "\t",
     "minbucket", "\t",     
     "ganancia_promedio", "\n")

# ---------------------------------------------------------------------------
# De un alumno de otro posgrado:
#
# Llegué a estos:
# cp = -1 <--- este parece no mover mucho las cosas, salvo que sea positivo, y ahí si se muere todo...
# minsplit = 850
# minbucket = 335
# maxdepth = 6 <--- este parece ser el mejor
# ---------------------------------------------------------------------------

#itero por los loops anidados para cada hiperparametro
for( v_cp  in  c(-0.09)  )
{
for( v_minbucket  in  c(303)  )
{
for( vmax_depth  in  c( 5, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6, 6.1, 6.2, 6.3, 6.4, 6.5))
{
for( vmin_split  in  c(2000))
{

  #notar como se agrega
  param_basicos  <- list( "cp"        =  v_cp,        # itero entre 0 y -1 con un p de .2
                          "minsplit"  =  vmin_split,  # mínima cantidad de registros en un nodo para hacer el split,
                          "minbucket" =  v_minbucket, # mínima cantidad de registros en una hoja
                          "maxdepth"  =  vmax_depth ) # profundidad máxima del árbol

  #Un solo llamado, con la semilla 17
  ganancia_promedio  <- ArbolesMontecarlo( ksemillas,  param_basicos )

  #escribo los resultados al archivo de salida
  cat(  file=archivo_salida,
        append= TRUE,
        sep= "",
        vmax_depth, "\t",
        vmin_split, "\t",
        v_cp, "\t",
        v_minbucket, "\t",      
        ganancia_promedio, "\n"  )

}
}
}
}

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,615688,32.9,1214197,64.9,1214197,64.9
Vcells,1151832,8.8,30998412,236.5,35195482,268.6
